# Домашняя работа по регуляризации и оптимизации

Ниже приводится корпус данных с двумя метками: 1 и -1. К данным применяется линейная модель классификации:

$f(x, \theta) = x_1 \theta_1 + x_2 \theta_2 + \theta_3.$

Предлагается подобрать параметры $\theta$ минимизируя следующую функцию ошибки:

$\mathcal{L}(\theta) = 0.1 \|\theta\|^2 + \frac{1}{N}\sum\limits_{i=1}^N \max(0, 1 - y_i f(x_i, \theta)).$

Для оптимизации предлагается использовать метод градиентного спуска с 1000 шагами размера $0.1$ из начальной точки $(1, 1, 0)$.

### Я как-то увлекся и начал писать на английском. 

Сбивающие с толку обозначения переменных $x_i$, так как в функции $f$ индексация обозначает элемент столбца $\begin{pmatrix} x_1 \\ x_2 \end{pmatrix}$, а в функции потерь $x_i$ обозначает номер пакета данных, т.е. номер самого столбца из двух иксов. В качестве обозначения столбца будем использовать $X_j$, а вместо $y_i$ использовать $Y_j$.

In [ ]:
import numpy as np
import yaml

In [ ]:
X1 = np.array([0, 1, 1, -0.5, 0], dtype='float64')
X2 = np.array([1, 1, 0, 0.5, -0.5], dtype='float64')
X3 = np.ones(5, dtype='float64') # for convience
X = np.array(list(zip(X1, X2, X3)))
Y = np.array([1, 1, 1, -1, -1], dtype='float64')

Нам будет удобнее считать в коде, что существует $x_3$ для каждого сампла, всегда равный 1. \\
Оно не будет менять результат.

In [ ]:
def f(x, theta):
  return x[0]*theta[0] + x[1]*theta[1] + x[2]*theta[2]

In [ ]:
def loss(theta, X, Y):
  N = len(X)
  max_arr = np.array([max(0, 1 - Y[i] * f(X[i], theta)) for i in range(N)])
  mean = (1/N) * np.sum(max_arr)
  return 0.1 * np.linalg.norm(theta)**2 + mean

In [ ]:
theta_start = np.array([1, 1, 0], dtype='float64')
loss(theta_start, X, Y)

# Считаем градиент лосса
$$\mathfrak{L}(\theta) = 0.1 ||\theta||^2 + \frac{1}{N} \sum^N_{i=1} \max(0, 1-Y_i f(X_i, \theta))  $$

$$f(x, \theta) = x_1 \theta_1 + x_2 \theta_2 + \theta_3 $$

We should compute gradient in respect to θ. 

Let $g(z(\theta)) = max(0, 1 - y z(\theta)), \ \ z(\theta) = f(X, \theta)$ \\

First, compute $z(\theta)$ gradient: \\
$$\frac{∂z}{∂θ_1} = x_1$$
$$\frac{∂z}{∂θ_2} = x_2$$
$$\frac{∂z}{∂θ_3} = 1$$
$$\nabla z = \begin{pmatrix} x_1 \\ x_2 \\ 1 \end{pmatrix}$$

Using chain rule:
$$\frac{∂}{∂θ}g(z(\theta)) = \frac{∂g}{∂z}\frac{∂z}{∂θ}$$

Then \\
$$\xi^i := \frac{\partial g(z(\theta))}{∂\theta_i} = 
\begin{cases}
  0, & y z > 1 \\
  - y x_i, & y z < 1
\end{cases}$$

(assuming $x_3$ = 1) \\
$-y$ comes from g's deriative, $x_i$ is from inner z's deriative.

Then compute mean over all data packs.

$$\frac{∂}{∂θ_i}\frac{1}{N} \sum^n_{j=1} \max(0, 1-Y_j f(X_j, \theta_i)) = \frac{1}{N} \sum_{j=1}^N \xi^i_j = \frac{1}{N} \sum_{j=1}^N \begin{cases} 0 & Y_j f(X_j, \theta) > 1 \\ - Y_j X_{ji}, & Y_j f(X_j, \theta) < 1\end{cases}$$

There $X_{ji}$ means $x_i$ from $X_j$ data column.

Let's find $||\theta||^2$ norm's deriative:
$$||\theta||^2 = \theta_1^2 + \theta_2^2 + \theta_3^2 $$

$$\frac{\partial ||\theta||^2}{∂θ_i} = 2\theta_i$$
$$\nabla ||\theta||^2 = \begin{pmatrix} 2θ_1 \\ 2θ_2 \\ 2θ_3 \end{pmatrix}$$

And finally
$$\frac{\partial \mathfrak{L}}{∂\theta_i} = 0.2 \theta_i + \frac{1}{N} \sum_{j=1}^N \xi^i_j$$

In [ ]:
theta = np.array([1, 1, 0], dtype='float64')

In [ ]:
def step(theta, X, Y, lr):
  grad = np.zeros(3)
  for i in range(len(theta)):
    grad[i] = 0.2 * theta[i]

    N = len(X)
    for j in range(N):
      der_mean = 0
      if (Y[j] * f(X[j], theta) < 1):
        der_mean += Y[j] * X[j][i]
      grad[i] -= (1/N) * der_mean

  return lr * grad

In [ ]:
def grad_descent(theta, X, Y, steps=10000, lr=0.1):
  for _ in range(steps):
    theta -= step(theta, X, Y, lr)

In [ ]:
grad_descent(theta, X, Y)

In [ ]:
def func(X, theta):
    theta = np.asarray(theta)
    return (X * theta[:2]).sum(axis=-1) + theta[2]

print("Prediction:", func(list(zip(X1, X2)), theta))
print("Loss:", loss(theta, X, Y))

with open("submission.yaml", "w") as fp:
    yaml.safe_dump({"tasks": [{"task1": {"answer": theta.tolist()}}]}, fp)